In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Glioblastoma"
cohort = "GSE226976"

# Input paths
in_trait_dir = "../../input/GEO/Glioblastoma"
in_cohort_dir = "../../input/GEO/Glioblastoma/GSE226976"

# Output paths
out_data_file = "../../output/preprocess/Glioblastoma/GSE226976.csv"
out_gene_data_file = "../../output/preprocess/Glioblastoma/gene_data/GSE226976.csv"
out_clinical_data_file = "../../output/preprocess/Glioblastoma/clinical_data/GSE226976.csv"
json_path = "../../output/preprocess/Glioblastoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Combined oncolytic adenovirus DNX-2401 and anti-PD-checkpoint inhibitor pembrolizumab for recurrent glioblastoma"
!Series_summary	"Gene expression data for samples included in this trial of DNX2401 and pembrolizumab for recurrent glioma"
!Series_overall_design	"Observational cohort"
!Series_overall_design	"CAPTIVE investigators"
Sample Characteristics Dictionary:
{0: ['sample type: recurrent glioma']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Callable, Optional, Dict, Any

# Analysis based on the given information

# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains "Gene expression data for samples included in this trial".
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# From the sample characteristics dictionary, we only have one key (0) with 'sample type: recurrent glioma'

# 2.1 Data Availability
# For trait (Glioblastoma):
# The sample type is described as "recurrent glioma", which is related to but not exactly Glioblastoma
# However, from the title, we know this is a study on "recurrent glioblastoma"
trait_row = 0  # The key where we can infer the trait (glioblastoma) status

# For age and gender:
# No information about age or gender is provided in the sample characteristics
age_row = None
gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(value: str) -> int:
    """Convert sample type to binary trait indicator for Glioblastoma."""
    if value is None:
        return None
    
    # Extract the value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    else:
        value = value.strip().lower()
    
    # From the title, we know this is about recurrent glioblastoma
    # The sample type mentions "recurrent glioma" which is likely referring to glioblastoma in this context
    if 'glioma' in value or 'glioblastoma' in value:
        return 1
    else:
        return 0

def convert_age(value: str) -> float:
    """Convert age value to float (not used in this dataset as age data is not available)."""
    return None

def convert_gender(value: str) -> int:
    """Convert gender value to binary (not used in this dataset as gender data is not available)."""
    return None

# 3. Save Metadata
# Check if trait data is available (trait_row is not None)
is_trait_available = trait_row is not None

# Save initial filtering information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we need to extract clinical features
if trait_row is not None:
    try:
        # Create directory for output if it doesn't exist
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Assuming clinical_data is already available from a previous step
        # We cannot proceed if it's not defined
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the extracted features
        preview = preview_df(selected_clinical_df)
        print("Preview of selected clinical features:")
        print(preview)
        
        # Save the clinical data
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error extracting clinical features: {e}")


Preview of selected clinical features:
{'GSM7089240': [1.0], 'GSM7089241': [1.0], 'GSM7089242': [1.0], 'GSM7089243': [1.0], 'GSM7089244': [1.0], 'GSM7089245': [1.0], 'GSM7089246': [1.0], 'GSM7089247': [1.0], 'GSM7089248': [1.0], 'GSM7089249': [1.0], 'GSM7089250': [1.0], 'GSM7089251': [1.0], 'GSM7089252': [1.0], 'GSM7089253': [1.0], 'GSM7089254': [1.0], 'GSM7089255': [1.0], 'GSM7089256': [1.0], 'GSM7089257': [1.0], 'GSM7089258': [1.0], 'GSM7089259': [1.0], 'GSM7089260': [1.0], 'GSM7089261': [1.0], 'GSM7089262': [1.0], 'GSM7089263': [1.0], 'GSM7089264': [1.0], 'GSM7089265': [1.0], 'GSM7089266': [1.0], 'GSM7089267': [1.0], 'GSM7089268': [1.0], 'GSM7089269': [1.0], 'GSM7089270': [1.0], 'GSM7089271': [1.0], 'GSM7089272': [1.0], 'GSM7089273': [1.0], 'GSM7089274': [1.0], 'GSM7089275': [1.0], 'GSM7089276': [1.0], 'GSM7089277': [1.0], 'GSM7089278': [1.0], 'GSM7089279': [1.0], 'GSM7089280': [1.0], 'GSM7089281': [1.0], 'GSM7089282': [1.0], 'GSM7089283': [1.0], 'GSM7089284': [1.0], 'GSM7089285': [

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 61
Header line: "ID_REF"	"GSM7089240"	"GSM7089241"	"GSM7089242"	"GSM7089243"	"GSM7089244"	"GSM7089245"	"GSM7089246"	"GSM7089247"	"GSM7089248"	"GSM7089249"	"GSM7089250"	"GSM7089251"	"GSM7089252"	"GSM7089253"	"GSM7089254"	"GSM7089255"	"GSM7089256"	"GSM7089257"	"GSM7089258"	"GSM7089259"	"GSM7089260"	"GSM7089261"	"GSM7089262"	"GSM7089263"	"GSM7089264"	"GSM7089265"	"GSM7089266"	"GSM7089267"	"GSM7089268"	"GSM7089269"	"GSM7089270"	"GSM7089271"	"GSM7089272"	"GSM7089273"	"GSM7089274"	"GSM7089275"	"GSM7089276"	"GSM7089277"	"GSM7089278"	"GSM7089279"	"GSM7089280"	"GSM7089281"	"GSM7089282"	"GSM7089283"	"GSM7089284"	"GSM7089285"	"GSM7089286"	"GSM7089287"
First data line: "A2M"	19.9968671	17.78207708	19.24881111	16.87933922	19.11810162	17.50234769	18.81256357	18.62195758	18.58962005	18.90626509	16.81346241	18.67700609	16.98598829	17.7329649	19.13751049	17.18096293	18.43889199	19.40324519	19.17214276	20.42669958	18.92025434	18.53633754	20.33795161	18.71954924	16.96514146	18.7

### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers in the gene expression data
# Looking at the sample index from the previous step:
# Index(['A2M', 'ABCF1', 'ACVR1C', 'ADAM12', 'ADGRE1', 'ADM', 'ADORA2A', 'AKT1',
#       'ALDOA', 'ALDOC', 'ANGPT1', 'ANGPT2', 'ANGPTL4', 'ANLN', 'APC', 'APH1B',
#       'API5', 'APLNR', 'APOE', 'APOL6'],
#      dtype='object', name='ID')

# These identifiers (A2M, ABCF1, etc.) are standard human gene symbols
# A2M = Alpha-2-Macroglobulin
# ABCF1 = ATP Binding Cassette Subfamily F Member 1
# AKT1 = AKT Serine/Threonine Kinase 1
# etc.

# Therefore, no mapping is required
requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# 1. First process the gene expression data from the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Extract gene expression data from the matrix file
gene_data = get_genetic_data(matrix_file)
print(f"Original gene data shape: {gene_data.shape}")
print(f"Sample gene symbols before normalization: {list(gene_data.index[:10])}")

# Normalize gene symbols using NCBI Gene database
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print(f"Sample gene symbols after normalization: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load clinical data
try:
    clinical_df = pd.read_csv(out_clinical_data_file, index_col=0)
    print(f"Clinical data shape: {clinical_df.shape}")
    print("Clinical data preview:")
    print(clinical_df.head())
except FileNotFoundError:
    print(f"Error: Clinical data file not found at {out_clinical_data_file}")
    # This is a critical error since we need clinical data to proceed
    raise

# 3. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, first 5 columns):")
if linked_data.shape[1] >= 5:
    print(linked_data.iloc[:5, :5])
else:
    print(linked_data.head())

# 4. Identify the trait column by inspecting the linked data
trait_col = clinical_df.index[0]  # The trait column will match the index in clinical_df
print(f"Identified trait column: {trait_col}")

# Handle missing values
print("\nMissing values before handling:")
print(f"  Trait ({trait_col}) missing: {linked_data[trait_col].isna().sum()} out of {len(linked_data)}")
if 'Age' in linked_data.columns:
    print(f"  Age missing: {linked_data['Age'].isna().sum()} out of {len(linked_data)}")
if 'Gender' in linked_data.columns:
    print(f"  Gender missing: {linked_data['Gender'].isna().sum()} out of {len(linked_data)}")

gene_cols = [col for col in linked_data.columns if col not in [trait_col, 'Age', 'Gender']]
if gene_cols:
    print(f"  Genes with >20% missing: {sum(linked_data[gene_cols].isna().mean() > 0.2)}")
    print(f"  Samples with >5% missing genes: {sum(linked_data[gene_cols].isna().mean(axis=1) > 0.05)}")

cleaned_data = handle_missing_values(linked_data, trait_col)
print(f"Data shape after handling missing values: {cleaned_data.shape}")

# 5. Evaluate bias in trait and demographic features
is_trait_biased = False
if len(cleaned_data) > 0:
    trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait_col)
    is_trait_biased = trait_biased
else:
    print("No data remains after handling missing values.")
    is_trait_biased = True

# 6. Final validation and save
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=len(normalized_gene_data) > 0, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=cleaned_data,
    note=f"Dataset contains gene expression data for {trait} analysis."
)

# 7. Save if usable
if is_usable and len(cleaned_data) > 0:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    cleaned_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable or empty and was not saved")

Original gene data shape: (770, 48)
Sample gene symbols before normalization: ['A2M', 'ABCF1', 'ACVR1C', 'ADAM12', 'ADGRE1', 'ADM', 'ADORA2A', 'AKT1', 'ALDOA', 'ALDOC']
Gene data shape after normalization: (762, 48)
Sample gene symbols after normalization: ['A2M', 'ABCF1', 'ACVR1C', 'ADAM12', 'ADGRE1', 'ADM', 'ADORA2A', 'AKT1', 'ALDOA', 'ALDOC']
Normalized gene data saved to ../../output/preprocess/Glioblastoma/gene_data/GSE226976.csv
Clinical data shape: (1, 47)
Clinical data preview:
            GSM7089241  GSM7089242  GSM7089243  GSM7089244  GSM7089245  \
GSM7089240                                                               
1.0                1.0         1.0         1.0         1.0         1.0   

            GSM7089246  GSM7089247  GSM7089248  GSM7089249  GSM7089250  ...  \
GSM7089240                                                              ...   
1.0                1.0         1.0         1.0         1.0         1.0  ...   

            GSM7089278  GSM7089279  GSM7089280  

Quartiles for '1.0':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1.0
Max: 1.0
The distribution of the feature '1.0' in this dataset is severely biased.



Data was determined to be unusable or empty and was not saved
